# 0 摘要

传统强化学习中，Agent面临着一些困难：
- 从高维感知环境的输入中提取有效的环境表达（Representations of the environment）
- 使用这些表达去总结概括过去经验，从而应对新的环境

过去强化学习的应用受限于某些领域，例如可以人工构造特征的领域或者某些具有完全可观察的、低维的状态空间领域下。

提出利用DNN去训练新的智能体Agent的方式，称为**（deep Q-network）**，能够使用端到端的强化学习方式直接从高维输入习得最优策略。

DQN在Atari经典的2600个游戏上经过测试，取得了超越过去所有算法性能的效果。通过接收像素与游戏分数作为输入，可以取得非常好的结果。

# 1 不足与改进

我们假设Agent面临的任务是通过观察(observations)、动作(actions)以及奖励(rewards)的序列来与环境交互，即任务可抽象为以下序列：

$$(s_0,a_0,r_1,s_1,a_1,\dots,r_T,s_T,a_T)$$

Agent目标是以某一种方式选择actions来最大化累积未来奖励(cumulative future reward)。本文中，作者使用深度卷积神经网络来近似最优的action-value function：

$$Q^*(s,a)=\max_\pi\mathbb{E}[r_t+\gamma r_{t+1} + \gamma^2 r_{t+2}+\dots +|s_t=s, a_t=a, \pi]$$

当使用非线性激活函数近似来表达action-value(Q) function时，强化学习的表现是不稳定且容易发散的，原因如下：

- 观察序列之间的相关性（序列之间不够独立，只要Q发生一点变化，就会影响到当前的action选择，从而影响后续的数据分布）
- action-value(也称Q值)和目标值TD Target($r+\gamma\max_{a'}Q(s',a')$)之间具有相关性

现存的方法例如neural fitted Q-iteration可以解决这个问题，但不够高效。因此作者提出两种新的关键idea来解决上述问题：

- **经验回放(Experience Replay)**
- **Q值的周期性迭代更新(Periodically Updated)**(或被称为Fixed Q-Targets)

## 1.1 符号定义

- 整个数据集$D_t=\{e_1,\dots,e_t\}$
- Agent在时刻t下的经验$e_t=(s_t,a_t,r_t,s_{t+1})$，其中$e_t\in

### 1 预处理

1. 输入：210 × 160像素点，每个像素点的取值有128个
2. 静态化图片。由于游戏中会有闪烁块，其本质上是通过在连续奇偶帧变换像素值实现闪烁，因此对连续的两帧(frame)图片在对应像素点位置取最大，获得静态图片；
3. 提取图片亮度Y Channel（忽略颜色），并切割为84×84的大小
4. 将当前帧前最近且连续的m个帧图像进行堆叠（实验中使用m=4)

### 2 模型结构

1. 输入：预处理后的84×84×4的image
2. conv_layer_1: num_filter=32, size=(8, 8)，stride=4, activation=relu
3. conv_layer_2: num_filter=64, size=(4, 4), stride=2, activation=relu
4. conv_layer_3: num_filter=64, size=(3, 3), stride=1, activation=relu
5. fc_layer: hidden_units=512, activation=relu
6. output_layer: hidden_units=action_space.n, activation=None

### 3 算法

### 3.1 Experience Replay

将Agent多个episodes的经验存储为一个数据集$D_t$，每次训练网络时候从其中sample一个样本进行参数更新。

好处：
- 一条样本会被采样多次用来参数更新，数据效率增加
- 由于序列样本之间的相关性，直接从原始序列样本中学习是低效率的；随机化采样可以避免相关性，减少更新参数的方差（？）
- 

不足：
- 由于采用固定大小的Replay Buffer，只保留最近N个Experience，Buffer中直接进行按顺序覆盖，因此不区分一些重要的experience
- 其次采用均匀分布抽样experience也没有考虑一些重要的经验

1. 初始化操作
    - 初始化回访缓存(Replay Memory)D的容量为N<br>
    - 初始化action-value函数Q为随机权重$\theta$<br>
    - 初始化target action-value函数$\hat{Q}$的权重$\theta^-=\theta$<br>

2. 遍历每一个Episode(1至M)
    - 初始化序列$s_1=\{x_1\}$，并将序列处理为$\phi_1=\phi(s_1)$(这里的具体操作是将图像进行静态化以及切割)<br>
    For t = 1 to T do<br>
        - 采用$\epsilon-greedy$去选择动作$a_t$
        - 执行动作$a_t$并记录返回的奖励$r_t$和下一个状态$x_{t+1}$
        - 

![](https://raw.githubusercontent.com/NELSONZHAO/paper-learning-and-notes/master/%E5%BC%BA%E5%8C%96%E5%AD%A6%E4%B9%A0/pictures/dqn-nature-1.png)

# 2 相关工作

# 3 介绍